<a href="https://colab.research.google.com/github/NikolasGialitsis/LDA2vec/blob/master/lda2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NikolasGialitsis/LDA2vec

Cloning into 'LDA2vec'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 170 (delta 34), reused 0 (delta 0), pack-reused 112
Receiving objects: 100% (170/170), 13.57 MiB | 6.50 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
!cp -r /content/drive/My\ Drive/covid_embeddings.zip /content
drive.flush_and_unmount()

## Setup and Preprocessing

Install Dependencies



In [20]:
#!git clone https://github.com/maxent-ai/lda2vec

!pip install spacy
!pip install jellyfish
#!pip install -r /content/lda2vec/requirements.txt 
!pip install pylda2vec


In [21]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


Import Libraries

In [22]:
%cd /content/LDA2vec
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec
from lda2vec import preprocess, Corpus
#changed Preprocess.py line 'nlp = spacy.load("en_core_web_sm")' to solve error en not found
#and removed the import ... as en line 

import logging
logging.basicConfig()
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from lda2vec import preprocess, Corpus

/content/LDA2vec


**CUDA** usage\
 *change Notebook Settings 'Hardware Acceleration' to 'GPU' to run*

In [23]:
gpu_id = int(os.getenv('CUDA_GPU', 0))
cuda.get_device(gpu_id).use()
print("Using GPU:" + str(gpu_id))

Using GPU:0


In [24]:
!unzip /content/LDA2vec/covid19_data.zip

Archive:  /content/LDA2vec/covid19_data.zip
replace db_entries.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: db_entries.json         


**Fetch Data**\
Set Stopwords
>>**DONE**: replace texts with covid19 collection

In [0]:
import json
with open('/content/LDA2vec/db_entries.json') as f:
  data = json.loads(f.read())
titles = []
abstracts = []
concats = []
with open('documents', 'a') as out:
  
  for entry in data:
    concat = (entry['joint_text']).split('------------------------------')  
    title = concat[0]
    abstract = concat[1]
    #print('title:',title)
    #print('abstract',abstract[:100])
    
    concats.append(concat[0] + ' ' + concat[1])
    titles.append(title)
    out.write(concat[0] + ' ' + concat[1])
    abstracts.append(abstract)



Pre-processing

*   Lowercase
*   Remove stopwords from nltk
*   Remove punctuation



In [26]:
import nltk 
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
texts = []
for text in concats.copy():
  word_tokens = word_tokenize(text) 
  #remove stopwords
  text = ' '.join([w.lower() for w in word_tokens if not w.lower() in stop_words and len(w) >= 3]) 
  #remove punctuation
  #bioclean    = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'", '').strip().lower()).split()

  text = ''.join([w for w in text if w not in string.punctuation])
  print(text[:100])
  texts.append(text)
    


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2019 novel coronavirus infection gastrointestinal tract since end december 2019 cluster patients pne
several suggestion operation colorectal cancer outbreak corona virus disease china pneumonia caused 
potential presymptomatic transmission sarscov2 zhejiang province china 2020 report 2family cluster p
straining system novel coronavirus covid19 preparedness concomitant disasters weeks first confirmed 
risk transportation 2019 novel coronavirus disease wuhan cities china january 2020 china quarantined
advances research cytokine storm mechanism induced corona virus disease 2019 corresponding immunothe
case 2019 novel coronavirus pregnant woman preterm delivery presented case 30week pregnant woman cov
diagnostic therapeutic strategies lung cancer patients outbreak

Preprocessing


*   set max word num per document
*   remove stopwords (aka. words in 'bad' list above)



In [0]:
# Preprocess data
max_length = 10000   # Limit of 10k words per document

Calculate tokens (list) and vocabulary (dictionary)\
OR\
Load them if already calculated

In [28]:
'''if a model is already trained,
skip to the cell loading vocab,corpus,flattened and doc_ids'''
tokens, vocab = preprocess.tokenize(texts, max_length, merge=False,n_threads=4)
'''LOAD IF ALREADY COMPUTED'''
#tokens = np.load("tokens.npy")
#vocab = np.load("vocab.npy")

'LOAD IF ALREADY COMPUTED'

run next line only if loaded

In [0]:
#vocab = vocab.tolist()

Bag-of-Words and Normalization
* remove really rare words
* downsample frequent words
* remove word gaps for words not in the vocabulary 
* flatten

In [0]:
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=4)
# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)

Pre-trained Word Vectors\
**TODO**: add pre-trained words when given

In [31]:
!unzip /content/covid_embeddings.zip

Archive:  /content/covid_embeddings.zip
   creating: covid_embeddings/
  inflating: covid_embeddings/covid_19_w2v_embeds_30.model  
  inflating: covid_embeddings/covid_19_w2v_embeds_30.model.trainables.syn1neg.npy  
  inflating: covid_embeddings/covid_19_w2v_embeds_30.model.wv.vectors.npy  


In [32]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
#!pip install gensim
assert flattened.min() >= 0
# Fill in the pretrained word vectors
n_dim = 30

import numpy as np
pretrained_filename= '/content/LDA2vec/covid_embeddings/covid_19_w2v_embeds_30.model'
vectors, s, f = corpus.compact_word_vectors(vocab, pretrained_filename,top=10027)
model = Word2Vec.load(pretrained_filename)
print(model.wv.syn0)

print(vectors)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[[ 1.9563258  -3.2772563   5.520347   ...  2.000226    1.9250718
  -2.3365216 ]
 [-0.18127924 -4.715032    6.332104   ... -0.73551935 -1.0634457
  -1.0868711 ]
 [-1.3495435  -0.48675144  1.5027217  ... -0.5184036   0.775794
  -1.9161127 ]
 ...
 [-0.01775365  0.11235748  0.16242264 ...  0.04308028  0.20860669
   0.56190693]
 [-0.22353493  0.14323027 -0.04657213 ... -0.2889772   0.01895762
   0.14997898]
 [ 0.17119046  0.09018349 -0.38958758 ...  0.2828389   0.3467089
   0.01221444]]
[[ 1.0361091   0.35070425  0.11374466 ... -1.7660041   1.758867
  -0.70263076]
 [ 1.119833    1.4275104   0.0195723  ...  0.6124658  -0.20076798
  -1.1631172 ]
 [-2.1628578   2.1892617   2.5789227  ... -1.5793166  -2.3099964
   1.0384628 ]
 ...
 [-0.641551   -0.49909088 -0.0076562  ...  1.0441295  -0.3340283
  -0.02855558]
 [ 0.06053621  0.07970229  0.00658272 ...  0.12321697  0.12195609
   0.32120553]
 [ 1.7871891  -0.07448457  1.556861   ... -0.7873218   1.5007665
  -0.13776147]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  if sys.path[0] == '':


Save all files (uncomment for vectors if pre-trained included)

In [0]:
# Save all of the preprocessed files
pickle.dump(vocab, open('vocab.pkl', 'wb'))
pickle.dump(corpus, open('corpus.pkl', 'wb'))
np.save("flattened", flattened)
np.save("doc_ids", doc_ids)
np.save("pruned", pruned)
np.save("bow", bow)


In [0]:
np.save("vectors", vectors)

In [35]:
print(tokens.shape)
print(vocab)

(744, 10000)
{0: '2019', 1: 'novel', 2: 'coronavirus', 3: 'infection', 4: 'gastrointestinal', 5: 'tract', 6: 'since', 7: 'end', 8: 'december', 9: 'cluster', 10: 'patients', 11: 'pneumonia', 12: 'unknown', 13: 'origin', 14: 'reported', 15: 'wuhan', 16: 'hubei', 17: 'province', 18: 'china', 19: 'feb', 20: '17th', 21: '2020', 22: 'statistical', 23: 'data', 24: 'show', 25: 'outbreak', 26: 'constitutes', 27: 'epidemic', 28: 'threat', 29: 'exponential', 30: 'increase', 31: 'reached', 32: '75114', 33: 'confirmed', 34: 'cases', 35: '2239', 36: 'deaths', 37: 'different', 38: 'sarscov', 39: 'severe', 40: 'acute', 41: 'respiratory', 42: 'syndrome', 43: 'merscov', 44: 'middle', 45: 'east', 46: 'initial', 47: 'presentations', 48: 'chief', 49: 'complain', 50: 'covid19', 51: 'symptoms', 52: 'call', 53: 'upon', 54: 'firstline', 55: 'medical', 56: 'staff', 57: 'cautious', 58: 'pay', 59: 'attention', 60: 'untypical', 61: 'especially', 62: 'area', 63: 'besides', 64: 'viral', 65: 'nucleic', 66: 'acids', 6

In [36]:
%cd /content/LDA2vec/lda2vec
#data_dir = os.getenv('data_dir', '../data/')
fn_vocab = 'vocab.pkl'
fn_corpus = 'corpus.pkl'
fn_flatnd = 'flattened.npy'
fn_docids = 'doc_ids.npy'
fn_vectors = 'vectors.npy'

/content/LDA2vec/lda2vec


##Defining Model Parameters



>> Parameters 
1.   Number of documents
2.   Number of unique words
3.   Dirichlet Prior 
4.   Number of topics
5.   Batchsize
6.   power for negative sampling 
7.   flag to include pre-trained or not
8.   sampling temperature
9.   number of dimensions of word vectors
10.  number of tokens in each document
11.  token frequencies



In [0]:
# Model Parameters
# Number of documents
n_docs = doc_ids.max() + 1
# Number of unique words in the vocabulary
n_vocab = flattened.max() + 1
# 'Strength' of the dircihlet prior; 200.0 seems to work well
clambda = 200.0
# Number of topics to fit
n_topics = int(os.getenv('n_topics', 10))
batchsize = 32
# Power for neg sampling
power = float(os.getenv('power', 0.75))
# Intialize with pretrained word vectors
pretrained = bool(int(os.getenv('pretrained', True)))
# Sampling temperature
temperature = float(os.getenv('temperature', 1.0))
# Number of dimensions in a single word vector
n_units = int(os.getenv('n_units', 30))
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_vocab]
# How many tokens are in each document
doc_idx, lengths = np.unique(doc_ids, return_counts=True)
doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
doc_lengths[doc_idx] = lengths
# Count all token frequencies
tok_idx, freq = np.unique(flattened, return_counts=True)
term_frequency = np.zeros(n_vocab, dtype='int32')
term_frequency[tok_idx] = freq

In [38]:
for key in sorted(locals().keys()):
    val = locals()[key]
    if len(str(val)) < 100 and '<' not in str(val):
        print(key, val)

Out {12: 'LOAD IF ALREADY COMPUTED', 28: 'LOAD IF ALREADY COMPUTED'}
_ LOAD IF ALREADY COMPUTED
_12 LOAD IF ALREADY COMPUTED
_28 LOAD IF ALREADY COMPUTED
__ LOAD IF ALREADY COMPUTED
___ 
__doc__ if a model is already trained,
skip to the cell loading vocab,corpus,flattened and doc_ids
__loader__ None
__name__ __main__
__package__ None
__spec__ None
_dh ['/content', '/content/LDA2vec', '/content/LDA2vec/lda2vec']
_exit_code 0
_i1 !git clone https://github.com/NikolasGialitsis/LDA2vec
_i11 max_length = 10000   # Limit of 10k words per document
_i13 #vocab = vocab.tolist()
_i15 !unzip /content/covid_embeddings.zip
_i17 from google.colab import drive
drive.mount('/content/drive')
_i18 !cp -r /content/drive/My\ Drive/covid_embeddings.zip /content
drive.flush_and_unmount()
_i2 from google.colab import drive
drive.mount('/content/drive')
_i21 !python -m spacy download en
_i24 !unzip /content/LDA2vec/covid19_data.zip
_i27 max_length = 10000   # Limit of 10k words per document
_i29 #vocab = voc

# Training the LDA2vec Model

In [0]:
model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                n_samples=15, power=power, temperature=temperature)

Add pretrained data

In [0]:

pretrained = True

'''
if os.path.exists('/content/lda2vec.hdf5'):
    print("Reloading from saved")
    serializers.load_hdf5("/content/lda2vec.hdf5", model)
#print(model.sampler.W.data)
'''
if pretrained:     
    model.sampler.W.data[:, :] = vectors[:n_vocab, :]

Adam Optimizer

In [0]:
model.to_gpu()
optimizer = O.Adam()
optimizer.setup(model)
clip = chainer.optimizer.GradientClipping(5.0)
optimizer.add_hook(clip)

In [0]:
j = 0
epoch = 0
fraction = batchsize * 1.0 / flattened.shape[0]
progress = shelve.open('progress.shelve')

Training with cuda


*   Measure coherence between topics every 100 iterations
*   this is for only one epoch
*   save the model
*   backpropagation style 



In [43]:
n_epochs = 1
for epoch in range(n_epochs):
    data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
    top_words = print_top_words_per_topic(data)
    if j % 100 == 0 and j > 100:
        coherence = topic_coherence(top_words)
        for j in range(n_topics):
            print(j, coherence[(j, 'cv')])
        kw = dict(top_words=top_words, coherence=coherence, epoch=epoch)
        progress[str(epoch)] = pickle.dumps(kw)
    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    np.savez('topics.pyldavis', **data)
    print(epoch)
    for d, f in utils.chunks(batchsize, doc_ids, flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy())
        print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
               "P:{prior:1.3e} R:{rate:1.3e}")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        logs = dict(loss=float(l), epoch=epoch, j=j,
                    prior=float(prior.data), rate=rate)
        print(msg.format(**logs))
        j += 1
serializers.save_hdf5("lda2vec.hdf5", model)

Top words in topic 0 must may could might would characteristics supplementary composition summarizes patterns
Top words in topic 1 threat arabia risk risks agent impact burden prevalence concern effectiveness
Top words in topic 2 day sem counts samples days staining images numbers count controls
Top words in topic 3 nucleic plaque carried protein kit assay extracted inhibits rna quantified
Top words in topic 4 described reveal play characterized show indicate supported discuss published identified
Top words in topic 5 amino nucleic epitopes residues codons position cleavage intervals peptide give
Top words in topic 6 play immune host affect contribute infect replication inhibit cellular hosts
Top words in topic 7 covid conserved protects genes homology coronaviruses strains shares identity mammals
Top words in topic 8 study cohort approach review cases algorithm diseases insights pneumonia patients
Top words in topic 9 assigned calculated higher selected carried used divided performed 

In [0]:
#!zip -r myzip.zip /content/lda2vec
#from google.colab import files
#files.download('myzip.zip')

In [46]:
npz = np.load(open('/content/LDA2vec/lda2vec/topics.pyldavis.npz', 'rb'))
dat = {k: v for (k, v) in npz.items()}
dat['vocab'] = dat['vocab'].tolist()
print(dat['vocab'])

['out_of_vocabulary', 'out_of_vocabulary', '<SKIP>', 'covid19', 'patients', 'coronavirus', 'disease', 'cases', '2019', 'china', 'sarscov2', 'clinical', 'health', 'respiratory', 'infection', 'outbreak', 'severe', 'novel', '2020', 'wuhan', 'epidemic', 'treatment', 'pneumonia', 'virus', 'acute', 'results', 'transmission', 'syndrome', 'control', 'confirmed', 'data', 'february', 'spread', 'risk', 'days', 'study', 'public', 'case', 'symptoms', 'methods', 'reported', 'care', 'infected', 'viral', 'number', 'management', 'may', 'pandemic', 'first', 'measures', 'group', 'countries', 'also', 'prevention', 'early', 'january', 'characteristics', 'caused', 'diagnosis', 'patient', 'findings', 'medical', 'emergency', 'new', '2019ncov', 'world', 'hubei', 'among', 'including', 'time', 'enditag', 'itag', 'hospital', 'december', 'chest', 'two', 'based', 'analysis', 'high', 'higher', 'features', 'using', 'current', 'positive', 'lung', 'fever', 'province', 'potential', 'epidemiological', 'global', 'period',

Print top 10 words for each topic

In [47]:
top_n = 100
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words

Topic 0 must may could might would characteristics supplementary composition summarizes patterns across changes receiver temporal evaluation performance demographic assessment experimental regarding insights pathological analysis visual optimization design status provides basic upon describes quality toward profile foot management shows consistency throughout right needs biosafety models dynamics towards demographics principles assessing aspects modeling safety optimizing physical insight various effects different analyses physiological using nutritional biochemical immunological dynamic differences stability reproductive distribution genetic parameters hygiene learning incorporating anesthetic concerning rather digestive evaluating three validation systems following usage considerations technical biological implications regardless assessments theoretical current makes special balance ethical distributions two understanding properties allows
Topic 1 threat arabia risk risks agent impac

In [0]:
serializers.save_hdf5("lda2vec.hdf5", model)
pickle.dump(model.mixture.weights.W.data, open('weights.pkl', 'wb'))
pickle.dump(model.mixture.factors.W.data, open('factors.pkl', 'wb'))
pickle.dump(model.sampler.W.data, open('sampler.pkl', 'wb'))


## Visualize Topics 
>> lower λ to get rare words appearing only on selected topics

In [49]:
!pip install pyLDAvis
import pyLDAvis
pyLDAvis.enable_notebook()
#warnings.filterwarnings('ignore')
import math

#dat['topic_term_dists'] = dat['topic_term_dists'] / dat['topic_term_dists'].sum(axis=1)[:, None]
prepared_data = pyLDAvis.prepare(dat['topic_term_dists'], dat['doc_topic_dists'], 
                                 dat['doc_lengths'] * 1.0, dat['vocab'], dat['term_frequency'] * 1.0, mds='tsne')

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 14.3MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=b21593d5a6d2b45c289e1c4c331cbf675bf5f0b1cf1f4990a300753131d0877e
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=db37b70c1816f903940780d45fac9d58731eb76df422f7ec882bd592f3880e9e
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [50]:
pyLDAvis.display(prepared_data)